In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [4]:
np.set_printoptions(precision=3, suppress=True)

In [7]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [8]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [9]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

In [10]:
raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [12]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [13]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'female' b'female' b'male']
age                 : [19.  16.  14.5 28.  35. ]
n_siblings_spouses  : [0 0 1 0 0]
parch               : [0 0 0 0 0]
fare                : [  7.65   26.     14.454  33.    512.329]
class               : [b'Third' b'Second' b'Third' b'Second' b'First']
deck                : [b'F' b'unknown' b'unknown' b'unknown' b'B']
embark_town         : [b'Southampton' b'Southampton' b'Cherbourg' b'Southampton' b'Cherbourg']
alone               : [b'y' b'y' b'n' b'y' b'y']


In [20]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'female' b'male' b'female' b'male']
age                 : [25.    0.75 28.   36.   62.  ]
n_siblings_spouses  : [0 2 1 1 0]
parch               : [0 1 0 0 0]
fare                : [ 7.65  19.258 16.1   26.    10.5  ]
class               : [b'Third' b'Third' b'Third' b'Second' b'Second']
deck                : [b'F' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'n' b'n' b'n' b'y']


In [26]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [29. 26.  4. 28. 18.]
n_siblings_spouses  : [0 0 1 0 0]
class               : [b'Third' b'Third' b'Second' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
alone               : [b'y' b'y' b'n' b'y' b'n']


In [30]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [34. 21. 35. 28. 52.]
n_siblings_spouses  : [0. 0. 0. 0. 1.]
parch               : [1. 0. 0. 0. 1.]
fare                : [23.     8.433  7.05  56.496 93.5  ]


In [31]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [34]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [38]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[28.    0.    0.    7.75]
 [19.    0.    0.    7.65]
 [65.    0.    0.   26.55]
 [35.    1.    0.   53.1 ]
 [28.    8.    2.   69.55]]

[0 0 0 1 0]


In [50]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'female' b'male']
age                 : [31. 28. 21. 27. 20.]
n_siblings_spouses  : [1 1 0 1 0]
parch               : [0 2 0 0 0]
fare                : [52.    23.45   8.05  13.858  9.846]
class               : [b'First' b'Third' b'Third' b'Second' b'Third']
deck                : [b'B' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Cherbourg' b'Southampton']
alone               : [b'n' b'n' b'y' b'n' b'y']


In [40]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [43]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [42]:
show_batch(packed_train_data)

sex                 : [b'male' b'female' b'female' b'male' b'female']
class               : [b'Second' b'Third' b'First' b'Second' b'First']
deck                : [b'F' b'unknown' b'C' b'unknown' b'B']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'n' b'y' b'y' b'y' b'n']
numeric             : [[  3.      1.      1.     26.   ]
 [ 18.      0.      0.      9.842]
 [ 35.      0.      0.    135.633]
 [ 27.      0.      0.     26.   ]
 [ 21.      2.      2.    262.375]]


In [53]:
!pip install pandas  # Required pandas
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

     |████████████████████████████████| 10.4MB 7.2MB/s eta 0:00:01    |██████████████████████████████▍ | 9.9MB 7.2MB/s eta 0:00:01
     |████████████████████████████████| 512kB 31.9MB/s eta 0:00:01


,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [54]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [55]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [56]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7fcc0dcc2378>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [57]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [58]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[53.   ,  2.   ,  0.   , 51.479],
       [66.   ,  0.   ,  0.   , 10.5  ],
       [35.   ,  0.   ,  0.   , 26.   ],
       [24.   ,  0.   ,  2.   , 16.7  ],
       [36.   ,  1.   ,  1.   , 24.15 ]], dtype=float32)>

In [59]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 1.868,  1.264, -0.479,  0.313],
       [ 2.907, -0.474, -0.479, -0.437],
       [ 0.429, -0.474, -0.479, -0.154],
       [-0.45 , -0.474,  2.043, -0.324],
       [ 0.509,  0.395,  0.782, -0.187]], dtype=float32)

In [60]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


In [61]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [62]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [63]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [64]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [72]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     1.     0.     0.     0.     0.     1.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     1.868  1.264
 -0.479  0.313  0.     1.   ]


In [78]:
example_batch

OrderedDict([('sex',
              <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'female', b'male', b'male', b'female', b'male'], dtype=object)>),
             ('class',
              <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'First', b'Second', b'Second', b'Third', b'Third'], dtype=object)>),
             ('deck',
              <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'C', b'unknown', b'unknown', b'G', b'unknown'], dtype=object)>),
             ('embark_town', <tf.Tensor: shape=(5,), dtype=string, numpy=
              array([b'Southampton', b'Southampton', b'Southampton', b'Southampton',
                     b'Southampton'], dtype=object)>),
             ('alone',
              <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'n', b'y', b'y', b'n', b'n'], dtype=object)>),
             ('numeric', <tf.Tensor: shape=(5, 4), dtype=float32, numpy=
              array([[53.   ,  2.   ,  0.   , 51.479],
                     [66.   ,  0.   ,  0.   , 10.5  ],
   

In [79]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [80]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [81]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 2s 12ms/step - loss: 0.5067 - accuracy: 0.7751
Epoch 2/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4152 - accuracy: 0.8246
Epoch 3/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3944 - accuracy: 0.8341
Epoch 4/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3974 - accuracy: 0.8278
Epoch 5/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3859 - accuracy: 0.8278
Epoch 6/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3703 - accuracy: 0.8357
Epoch 7/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3638 - accuracy: 0.8485
Epoch 8/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3585 - accuracy: 0.8453
Epoch 9/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3657 - accuracy: 0.8325
Epoch 10/20
126/126 [==============================] - 0s 4ms/step - loss: 0.3479 - accuracy: 0.842

In [82]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

     53/Unknown - 1s 10ms/step - loss: 0.4835 - accuracy: 0.8106

Test Loss 0.4834953469206702, Test Accuracy 0.810606062412262


In [83]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))



Predicted survival: 16.62%  | Actual outcome:  DIED
Predicted survival: 10.36%  | Actual outcome:  DIED
Predicted survival: 98.47%  | Actual outcome:  SURVIVED
Predicted survival: 79.75%  | Actual outcome:  DIED
Predicted survival: 64.53%  | Actual outcome:  DIED
